# Set Up

In [1]:
# import necessary packages
import pandas as pd
import numpy as np
from nilearn import surface
import usefulFunctions as uf
import os
import glob
from scipy.stats import ttest_rel

In [2]:
# falg to save csvs
save_csv = True

# set directories
base_dir = f'{os.path.dirname(os.getcwd())}/'
surf_dir = f'{base_dir}data/surface_meshes/'
label_dir = f'{base_dir}data/labels/native/sepTask/'
csv_dir = f'{base_dir}analysis/CSVs/'

# define subject list
subs = uf.subject_list(surf_dir)
#separate sub list into adults and children
id_cutoff = 1000 # split between child and adult ID #s
child_subs = [sub for sub in subs if int(sub.split('-')[1]) < id_cutoff]
adult_subs = [sub for sub in subs if int(sub.split('-')[1]) >= id_cutoff]

# Calculate Dice Similarity Coefficient (DSC)

In [3]:
# specify groups
sub_groups = [subs,child_subs,adult_subs]
group_names = ['all','children','adults']

In [4]:
# loop through ROIs to collect DSC

for group_indx, group in enumerate(sub_groups):
    dice_cos = []
    for sub in group:
        # grab any surface file for each participant
        surf = surface.load_surf_data(f'{base_dir}data/t_maps/native/allData/{sub}/{sub}_T-v-ALL_LH.curv')
    
        #grab label paths
        ob_labels = glob.glob(f'{label_dir}{sub}/{sub}_task-oneback_T-v-ALL_VWFA*_LH.label')
        fix_labels = glob.glob(f'{label_dir}{sub}/{sub}_task-fixation_T-v-ALL_VWFA*_LH.label')
    
        # continue with DSC caluclation on if participant has both labels
        if len(ob_labels)>0 and len(fix_labels)>0:
    
            # load labels
            vwfa_ob = uf.read_in_rois(ob_labels)
            vwfa_fix = uf.read_in_rois(fix_labels)
    
            # create surface masks
            vwfa_ob_mask = np.zeros(surf.shape)
            vwfa_fix_mask = np.zeros(surf.shape)
    
            #add binary indiciator in any vertext contianing the roi
            vwfa_ob_mask[vwfa_ob]=1
            vwfa_fix_mask[vwfa_fix]=1
    
            #calculate DSC
            intersection = np.logical_and(vwfa_ob_mask, vwfa_fix_mask)
            dice = 2. * intersection.sum() / (vwfa_ob_mask.sum() + vwfa_fix_mask.sum())
            dice_cos.append(dice)
        
    # print results
    print(group_names[group_indx])
    print(f'mean: {np.mean(dice_cos)}')
    print(f'sd: {np.std(dice_cos)}')
    print(f'se: {np.std(dice_cos) / np.sqrt(np.size(dice_cos))}')


all
mean: 0.46474833093922663
sd: 0.2616974784479382
se: 0.03127883132316466
children
mean: 0.42980497694184705
sd: 0.26028333768445366
se: 0.03447537027679461
adults
mean: 0.617961498466199
sd: 0.2071087552253124
se: 0.05744163358172187


# Calculate Size Difference

In [6]:
# define tasks
tasks = ['oneback','fixation']

# specify groups
sub_groups = [child_subs,adult_subs]
group_names = ['children','adults']

In [7]:
#create empty list to store data
size_data = []

# loop through ROIs to collect size
for group_indx, group in enumerate(sub_groups):
    dice_cos = []
    for sub in group:
        for task in tasks:
        
            #grab label paths
            labels = glob.glob(f'{label_dir}{sub}/{sub}_task-{task}_T-v-ALL_VWFA*_LH.label')
            
            # sum vertices for roi size
            if len(labels)>0:
                roi_size = len(uf.read_in_rois(labels))
            # if label doesn't exist then mark as size 0
            else:
                roi_size = 0
    
            # append data to list
            size_data.append({'sub': sub,
                                 'group': group_names[group_indx],
                                 'task': task,
                                 'roi_size': roi_size,
                                 'log_size': np.log10(roi_size+1)
                                })

# convert list to data frame
size_df = pd.DataFrame(size_data)

# save df
if save_csv:
    size_df.to_csv(f'{csv_dir}VWFA_size.csv',index=False)

In [33]:
# pivot table
df_wide = size_df.pivot_table(index=['sub','group'], columns='task', values=['roi_size','log_size'], fill_value=0)
df_wide = df_wide.reset_index()

# Flatten the MultiIndex columns
df_wide.columns = ['_'.join(filter(None, col)).strip() for col in df_wide.columns]

In [34]:
# specify group_names
group_names = ['all','children','adults']

# loop through groups to calculate t stats
for group in group_names:
    if group == 'all':
        test_results = ttest_rel(df_wide['log_size_oneback'],df_wide['log_size_fixation'])
    else:
        test_results = ttest_rel(df_wide.query('group==@group')['log_size_oneback'],df_wide.query('group==@group')['log_size_fixation'])
    
    # print results
    print(f'\n{group}')
    print(test_results)
    print(test_results.confidence_interval())


all
TtestResult(statistic=2.4164237826290575, pvalue=0.01758438691028531, df=95)
ConfidenceInterval(low=0.029178278323204382, high=0.29786966660528447)

children
TtestResult(statistic=2.3577396314980645, pvalue=0.02079866463464977, df=81)
ConfidenceInterval(low=0.028701384095988852, high=0.33901921096160315)

adults
TtestResult(statistic=0.5607766830293549, pvalue=0.5844860938289966, df=13)
ConfidenceInterval(low=-0.12668111815394945, high=0.21550354089769141)


# Calculate Selectivity Index

In [40]:
# load the psc data
psc_df = pd.read_csv(f'{csv_dir}sepTaskROIs_sepTaskMaps_meanPSC.csv')

#filter fo only categories of interest
psc_data = psc_df.query('category=="T" or category=="noText"')

# pivot wider for convience of SI calculation
pivoted_df = psc_data.pivot(columns=['category'],values=['mean_psc'],index=['sub','roi','space','roi_task','map_task'])
pivoted_df=pivoted_df.reset_index()

# reanme columns
pivoted_df.columns = ['_'.join(filter(None, col)).strip() for col in pivoted_df.columns]

# calcualte SI
pivoted_df['SI'] = ((1 + pivoted_df['mean_psc_T']) - (1 + pivoted_df['mean_psc_noText'])) / ((1 + pivoted_df['mean_psc_T']) + (1 + pivoted_df['mean_psc_noText']))

#save df
if save_csv:
    pivoted_df.to_csv(f'{csv_dir}VWFA_SI.csv',index=False)